## CNN ResNet 분류기 구현 testcode
### [3분 딥러닝 참고](https://github.com/keon/3-min-pytorch/blob/master/05-%EC%9D%B4%EB%AF%B8%EC%A7%80_%EC%B2%98%EB%A6%AC%EB%8A%A5%EB%A0%A5%EC%9D%B4_%ED%83%81%EC%9B%94%ED%95%9C_CNN/resnet.ipynb)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, models

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [5]:
# 하이퍼 파라미터
EPOCHS = 300
BATCH_SIZE = 128

In [9]:
# dataset loader
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                    train = True,
                    download = True,
                    transform = transforms.Compose([
                        transforms.RandomCrop(32, padding=4),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5),
                                             (0.5, 0.5, 0.5))])),
    batch_size = BATCH_SIZE, shuffle = True)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                    train = False,
                    transform = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5),
                                             (0.5, 0.5, 0.5))])),
    batch_size = BATCH_SIZE, shuffle = True)

Extracting ./.data\cifar-10-python.tar.gz to ./.data


In [10]:
# ResNet Model
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride = 1) :
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size = 3,
                               stride = stride, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size = 3,
                               stride = 1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes :
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size = 1,
                          stride = stride, bias = False),
                nn.BatchNorm2d(planes)
            )
            
    def forward(self, x) :
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
class ResNet(nn.Module) :
    def __init__(self, num_classes = 10):
        super(ResNet, self).__init__()
        self.in_planes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3,
                               stride = 1, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride = 1)
        self.layer2 = self._make_layer(32, 2, stride = 2)
        self.layer3 = self._make_layer(64, 2, stride = 2)
        self.linear = nn.Linear(64, num_classes)
        
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides :
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [11]:
model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr = 0.1,
                      momentum = 0.9, weight_decay = 0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 50, gamma = 0.1)

In [12]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output= model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [13]:
def evaluate(model, tets_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad() :
        for data, target in test_loader :
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            # batch 오차 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction = 'sum').item()
            
            # 가장 높은 값을 가진 인덱스를 예측값으로
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [14]:
# 실행
for epoch in range(1, EPOCHS + 1):
    scheduler.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] TEST Loss : {:.4f}, Accuracy: {:.2f}%'.format(
            epoch, test_loss, test_accuracy))

C:\Users\SM-PC\Anaconda3\envs\py364\lib\site-packages\torch\optim\lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1] TEST Loss : 1.4717, Accuracy: 45.60%
[2] TEST Loss : 1.3717, Accuracy: 54.05%
[3] TEST Loss : 1.0294, Accuracy: 64.03%
[4] TEST Loss : 1.1823, Accuracy: 62.65%
[5] TEST Loss : 0.8297, Accuracy: 71.83%
[6] TEST Loss : 0.7904, Accuracy: 73.51%
[7] TEST Loss : 0.7620, Accuracy: 73.99%
[8] TEST Loss : 0.9641, Accuracy: 67.82%
[9] TEST Loss : 0.8458, Accuracy: 71.70%
[10] TEST Loss : 0.6071, Accuracy: 79.09%
[11] TEST Loss : 1.3084, Accuracy: 64.21%
[12] TEST Loss : 0.7514, Accuracy: 75.83%
[13] TEST Loss : 0.8609, Accuracy: 73.29%
[14] TEST Loss : 0.6999, Accuracy: 76.30%
[15] TEST Loss : 1.0367, Accuracy: 69.03%
[16] TEST Loss : 0.8023, Accuracy: 75.08%
[17] TEST Loss : 0.6402, Accuracy: 78.35%
[18] TEST Loss : 0.7079, Accuracy: 76.43%
[19] TEST Loss : 1.0155, Accuracy: 70.10%
[20] TEST Loss : 0.6354, Accuracy: 78.52%
[21] TEST Loss : 0.7061, Accuracy: 76.37%
[22] TEST Loss : 0.6248, Accuracy: 78.14%
[23] TEST Loss : 0.5535, Accuracy: 81.25%
[24] TEST Loss : 0.8312, Accuracy: 73.53%
[

[195] TEST Loss : 0.2954, Accuracy: 91.13%
[196] TEST Loss : 0.2957, Accuracy: 91.07%
[197] TEST Loss : 0.2973, Accuracy: 90.97%
[198] TEST Loss : 0.2958, Accuracy: 90.97%
[199] TEST Loss : 0.2952, Accuracy: 91.12%
[200] TEST Loss : 0.2967, Accuracy: 91.01%
[201] TEST Loss : 0.2963, Accuracy: 91.02%
[202] TEST Loss : 0.2978, Accuracy: 91.04%
[203] TEST Loss : 0.2987, Accuracy: 90.98%
[204] TEST Loss : 0.2966, Accuracy: 91.00%
[205] TEST Loss : 0.2957, Accuracy: 91.07%
[206] TEST Loss : 0.2985, Accuracy: 91.04%
[207] TEST Loss : 0.2973, Accuracy: 91.06%
[208] TEST Loss : 0.2972, Accuracy: 91.07%
[209] TEST Loss : 0.2961, Accuracy: 90.97%
[210] TEST Loss : 0.2972, Accuracy: 91.12%
[211] TEST Loss : 0.2961, Accuracy: 90.89%
[212] TEST Loss : 0.2962, Accuracy: 91.06%
[213] TEST Loss : 0.2957, Accuracy: 91.08%
[214] TEST Loss : 0.2963, Accuracy: 91.09%
[215] TEST Loss : 0.2969, Accuracy: 91.07%
[216] TEST Loss : 0.2971, Accuracy: 91.10%
[217] TEST Loss : 0.2970, Accuracy: 91.15%
[218] TEST 